In [ ]:
! mamba install htop polars -y

In [2]:
from pathlib import Path

import xarray as xr
import fsspec
import dask.dataframe as dd
import dask
from tqdm import tqdm
import duckdb
import polars as pl
import numpy as np
import pandas as pd

In [3]:
DATA_DIR = Path("/data/protocols/retro/timeseries")

USGS_TIMESERIES = Path(DATA_DIR, "usgs_201*.parquet")

LOCAL_JOINED_FILEPATH = "/data/protocols/retro/timeseries/joined*.parquet"

LOCAL_JOINED_NWM20_2016 = "/data/protocols/retro/timeseries/joined_nwm20_retrospective_2016.parquet"
LOCAL_JOINED_NWM20_2017 = "/data/protocols/retro/timeseries/joined_nwm20_retrospective_2017.parquet"
LOCAL_JOINED_NWM20_2018 = "/data/protocols/retro/timeseries/joined_nwm20_retrospective_2018.parquet"

LOCAL_JOINED_NWM20_ALL = "/data/protocols/retro/timeseries/joined_nwm20_retrospective_201*.parquet"
LOCAL_JOINED_NWM21_ALL = "/data/protocols/retro/timeseries/joined_nwm21_retrospective_201*.parquet"

LOCAL_NWM20_2016 = "/data/protocols/retro/timeseries/nwm20_2016.parquet"
LOCAL_NWM20_2017 = "/data/protocols/retro/timeseries/nwm20_2017.parquet"
LOCAL_NWM20_2018 = "/data/protocols/retro/timeseries/nwm20_2018.parquet"

LOCAL_NWM21_2016 = "/data/protocols/retro/timeseries/nwm21_2016.parquet"
LOCAL_NWM21_2017 = "/data/protocols/retro/timeseries/nwm21_2017.parquet"
LOCAL_NWM21_2018 = "/data/protocols/retro/timeseries/nwm21_2018.parquet"

LOCAL_ZARR_JOINED_ALL = "/data/protocols/retro/timeseries/joined_nwm20_nwm21_2016_2018.zarr"
LOCAL_ZARR_JOINED_NWM20 = "/data/protocols/retro/timeseries/joined_nwm20_2016_2018.zarr"
LOCAL_ZARR_NWM20 = "/data/protocols/retro/timeseries/nwm20_2016_2018.zarr"

LOCAL_ZARR_JOINED_NWM21 = "/data/protocols/retro/timeseries/joined_nwm21_2016_2018.zarr"

# nrows = 1000

In [3]:
# query = "CREATE TYPE VARIABLE_NAME_DTYPE AS ENUM ('streamflow');"
# duckdb.sql(query)

# query = "CREATE TYPE MEASUREMENT_UNIT_DTYPE AS ENUM ('cms');"
# duckdb.sql(query)

# query = "CREATE TYPE CONFIGURATION_DTYPE AS ENUM ('usgs_gage_data');"
# duckdb.sql(query)

**Pivot timeseries parquet files**  (Note: Might be faster to do the pivot using polars?)

In [6]:
def load_dataframe_duckdb(filepath):
    """Load a parquet file(s) as a dataframe using duckdb"""
    query = f"""
        PIVOT(
            SELECT
                location_id,
                value_time,
                value,
            FROM
                read_parquet('{str(filepath)}')
        ) ON location_id USING first(value)
    ;"""
    return duckdb.sql(query).pl()

**Load with duckdb**

In [7]:
def load_dataframe_duckdb(filepath):
    """Load a parquet file(s) as a dataframe using duckdb"""
    query = f"""
        SELECT
            location_id,
            value_time,
            value,
        FROM
            read_parquet('{str(filepath)}')
    ;"""
    df = duckdb.sql(query).pl()

    return df

**Pivot joined timeseries parquet files with duckdb**

In [123]:
def load_primary_dataframe_duckdb(filepath):
    """Load a parquet file(s) as a dataframe using duckdb"""
    query = f"""
        PIVOT(
            SELECT
                primary_location_id,
                primary_value,
                value_time,
            FROM
                read_parquet('{str(filepath)}')
        ) ON primary_location_id USING first(primary_value)
    ;"""
    return duckdb.sql(query).pl()

def load_secondary_dataframe_duckdb(filepath):
    """Load a parquet file(s) as a dataframe using duckdb"""
    query = f"""
        PIVOT(
            SELECT
                secondary_location_id,
                secondary_value,
                value_time,
            FROM
                read_parquet('{str(filepath)}')
        ) ON secondary_location_id USING first(secondary_value)
    ;"""
    return duckdb.sql(query).pl()

**Load with duckdb, pivot joined timeseries with Polars**

In [4]:
def load_primary_dataframe_polars(filepath):
    """Load a parquet file(s) as a dataframe using duckdb"""
    query = f"""
        SELECT
            primary_location_id,
            primary_value,
            value_time,
        FROM
            read_parquet('{str(filepath)}')
        ORDER BY value_time
    ;"""
    df = duckdb.sql(query).pl()
    return df.pivot(values="primary_value", index="value_time", columns="primary_location_id")
    # return df

def load_secondary_dataframe_polars(filepath):
    """Load a parquet file(s) as a dataframe using duckdb"""
    query = f"""
        SELECT
            secondary_location_id,
            secondary_value,
            value_time,
        FROM
            read_parquet('{str(filepath)}')
        ORDER BY value_time
    ;"""
    df = duckdb.sql(query).pl()
    return df.pivot(values="secondary_value", index="value_time", columns="secondary_location_id")
    # return df

**General query head**

In [20]:
def load_both_dataframe_polars(filepath):
    """Load a parquet file(s) as a dataframe using duckdb"""
    query = f"""
        SELECT
            primary_location_id,
            secondary_location_id,
            primary_value,
            secondary_value,
            value_time,
        FROM
            read_parquet('{str(filepath)}')
        ORDER BY value_time
    ;"""
    df = duckdb.sql(query).pl()
    return df

def load_both_dict_numpy(filepath):
    """Load a parquet file(s) as a dataframe using duckdb"""
    query = f"""
        SELECT
            primary_location_id,
            secondary_location_id,
            primary_value,
            secondary_value,
            value_time,
        FROM
            read_parquet('{str(filepath)}')
        ORDER BY value_time
    ;"""
    np_dict = duckdb.sql(query).fetchnumpy()
    return np_dict

# Fetch the data

In [17]:
 #df.pivot(values="secondary_value", index="value_time", columns="secondary_location_id")

In [48]:
%%time
# df_pri = load_primary_dataframe_polars(LOCAL_JOINED_NWM20_ALL)
# df_sec = load_secondary_dataframe_polars(LOCAL_JOINED_NWM20_ALL)

both_df = load_both_dataframe_polars(LOCAL_JOINED_NWM21_ALL)
# both_dict = load_both_dict_numpy(LOCAL_JOINED_NWM20_ALL)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 2min 54s, sys: 3min 39s, total: 6min 34s
Wall time: 1min 19s


In [49]:
# both_dict["primary_value"].shape
df_pri = both_df.pivot(values="primary_value", index="value_time", columns="primary_location_id")
df_sec = both_df.pivot(values="secondary_value", index="value_time", columns="secondary_location_id")

# Single Timeseries

In [42]:
value_arr = df_ts.to_numpy()
value_arr = value_arr[:, 1:]
value_times = df_ts["value_time"].to_numpy()
location_ids = np.array(df_ts.columns)[1:]

coords_dict = {
    "location_id": location_ids,
    "value_time": np.array(value_times).astype("datetime64[ns]"),
    "reference_time": ("value_time", np.array(value_times).astype("datetime64[ns]")),
    "configuration": "nwm21"
}
dims_list = ["value_time", "location_id"]  

In [43]:
da_ts = xr.DataArray(data=value_arr, name="streamflow", dims=dims_list, coords=coords_dict) 
da_ts

<xarray.DataArray 'streamflow' (value_time: 8760, location_id: 7647)> Size: 536MB
array([[6.26609986e+02, 6.27069986e+02, 0.00000000e+00, ...,
        1.69999996e-01, 1.86999996e+00, 6.23759986e+02],
       [6.23839986e+02, 6.24299986e+02, 0.00000000e+00, ...,
        3.59999992e-01, 3.25999993e+00, 6.20999986e+02],
       [6.20139986e+02, 6.20589986e+02, 0.00000000e+00, ...,
        1.99999996e-01, 8.74999980e+00, 6.17299986e+02],
       ...,
       [6.41339986e+02, 6.41849986e+02, 0.00000000e+00, ...,
        1.69999996e-01, 4.64999990e+00, 6.38379986e+02],
       [6.37319986e+02, 6.37789986e+02, 0.00000000e+00, ...,
        2.29999995e-01, 3.07999993e+00, 6.34439986e+02],
       [6.32139986e+02, 6.32599986e+02, 0.00000000e+00, ...,
        1.89999996e-01, 1.96999996e+00, 6.29269986e+02]])
Coordinates:
  * location_id     (location_id) <U16 489kB 'nwm21-10010638' ... 'nwm21-9998...
  * value_time      (value_time) datetime64[ns] 70kB 2018-05-14T23:00:00 ... ...
    reference_time  (value_time) datetime64[ns] 70kB 2018-05-14T23:00:00 ... ...
    configuration   <U19 76B 'nwm21_retrospective'

In [44]:
ds = da_ts.to_dataset().chunk({"value_time": 8764, "location_id": 100}) 
ds = ds.assign_attrs(measurement_unit="cms")

In [45]:
# TODO: TEMP!
indx = 1

if indx == 0:
    ds.to_zarr("/data/protocols/retro/timeseries/nwm21_2016_2018.zarr", mode="w", consolidated=True) 
else:
    ds.to_zarr("/data/protocols/retro/timeseries/nwm21_2016_2018.zarr", mode="a", append_dim="value_time", consolidated=True)

In [24]:
ds = xr.open_dataset("/data/protocols/retro/timeseries/nwm20_2016_2018.zarr", engine="zarr")
ds

<xarray.Dataset> Size: 2GB
Dimensions:         (location_id: 7541, value_time: 26304)
Coordinates:
    configuration   <U19 76B ...
  * location_id     (location_id) <U16 483kB 'nwm20-10010638' ... 'nwm20-9998...
    reference_time  (value_time) datetime64[ns] 210kB ...
  * value_time      (value_time) datetime64[ns] 210kB 2016-12-07T18:00:00 ......
Data variables:
    streamflow      (value_time, location_id) float64 2GB ...
Attributes:
    measurement_unit:  cms

# Joined Timeseries

**Create separate primary and secondary data arrays, then concat**

**Primary data array**

In [50]:
sec_location_ids = np.array(df_sec.columns)[1:]
sec_location_ids

array(['nwm21-6163185', 'nwm21-6162981', 'nwm21-6163249', ...,
       'nwm21-19947688', 'nwm21-5736533', 'nwm21-13463913'], dtype='<U16')

In [51]:
pri_location_ids = np.array(df_pri.columns)[1:]
pri_location_ids

array(['usgs-01122500', 'usgs-01123000', 'usgs-011230695', ...,
       'usgs-07341500', 'usgs-08143500', 'usgs-05536357'], dtype='<U20')

In [52]:
indx = np.arange(pri_location_ids.size)

In [53]:
value_arr = df_pri.to_numpy()
value_arr = value_arr[:, 1:]
value_times = df_pri["value_time"].to_numpy()
# value_times = np.array(df_pri.columns[1:]) # transposed

coords_dict = {
    "location_index": indx,
    "value_time": np.array(value_times).astype("datetime64[ns]"),
    "reference_time": ("value_time", np.array(value_times).astype("datetime64[ns]")),
    "primary_location_id": ("location_index", pri_location_ids)
}
dims_list = ["value_time", "location_index"]   

In [54]:
da_pri = xr.DataArray(data=value_arr, name="streamflow", dims=dims_list, coords=coords_dict) 
da_pri

<xarray.DataArray 'streamflow' (value_time: 26280, location_index: 7408)> Size: 2GB
array([[1.71316910e+01, 1.41584229e+00, 1.61122856e+01, ...,
                   nan,            nan,            nan],
       [1.71316910e+01, 1.45265424e+00, 1.99067421e+01, ...,
                   nan,            nan,            nan],
       [1.72732754e+01, 1.48946607e+00, 3.05821934e+01, ...,
                   nan,            nan,            nan],
       ...,
       [4.89881439e+01, 3.51128888e+00, 6.22970619e+01, ...,
        2.32764478e+03,            nan, 3.08653617e+00],
       [4.84218063e+01, 3.51128888e+00, 5.83327026e+01, ...,
        2.32764478e+03,            nan, 3.05821943e+00],
       [4.81386375e+01, 3.48297215e+00, 5.72000275e+01, ...,
        2.32198145e+03,            nan, 2.97326875e+00]])
Coordinates:
  * location_index       (location_index) int64 59kB 0 1 2 3 ... 7405 7406 7407
  * value_time           (value_time) datetime64[ns] 210kB 2016-01-01 ... 201...
    reference_time       (value_time) datetime64[ns] 210kB 2016-01-01 ... 201...
    primary_location_id  (location_index) <U20 593kB 'usgs-01122500' ... 'usg...

In [55]:
# TEST
# test_id = "usgs-01163200"
# da_pri = da_pri.set_xindex("primary_location_id")
# da_pri.sel(primary_location_id=test_id).to_dataframe()

**Secondary data array**

In [56]:
# location_ids = np.array(df_sec.columns)[1:]
value_arr = df_sec.to_numpy()
value_arr = value_arr[:, 1:]
value_times = df_sec["value_time"].to_numpy()

coords_dict = {
    "location_index": indx,
    "value_time": np.array(value_times).astype("datetime64[ns]"),
    "reference_time": ("value_time", np.array(value_times).astype("datetime64[ns]")),
    "secondary_location_id": ("location_index", sec_location_ids)
}
dims_list = ["value_time", "location_index"]     

In [57]:
da_sec = xr.DataArray(data=value_arr, name="streamflow", dims=dims_list, coords=coords_dict) 
da_sec

<xarray.DataArray 'streamflow' (value_time: 26280, location_index: 7408)> Size: 2GB
array([[  19.60000038,    1.88      ,   22.97999954, ...,           nan,
                  nan,           nan],
       [  19.63999939,    1.81999993,   23.        , ...,           nan,
                  nan,           nan],
       [  19.68000031,    1.73000002,   23.03999901, ...,           nan,
                  nan,           nan],
       ...,
       [  41.56999969,    3.26999998,   57.56999969, ..., 1294.08996582,
                  nan,    1.5999999 ],
       [  41.21999741,    3.28999996,   56.98999786, ..., 1295.63000488,
                  nan,    1.5999999 ],
       [  40.88999939,    3.27999997,   56.41999817, ..., 1297.5300293 ,
                  nan,    1.5999999 ]])
Coordinates:
  * location_index         (location_index) int64 59kB 0 1 2 ... 7405 7406 7407
  * value_time             (value_time) datetime64[ns] 210kB 2016-01-01 ... 2...
    reference_time         (value_time) datetime64[ns] 210kB 2016-01-01 ... 2...
    secondary_location_id  (location_index) <U16 474kB 'nwm21-6163185' ... 'n...

In [58]:
# # TEST
# test_id = "usgs-01163200"
# da_sec = da_sec.set_xindex("seconar")
# da_sec.sel(primary_location_id=test_id).to_dataframe()

**Combine data arrays**

In [59]:
da_both = xr.concat([da_pri, da_sec], pd.Index(["primary_value", "secondary_value"], name="timeseries_name"))
da_both

<xarray.DataArray 'streamflow' (timeseries_name: 2, value_time: 26280,
                                location_index: 7408)> Size: 3GB
array([[[1.71316910e+01, 1.41584229e+00, 1.61122856e+01, ...,
                    nan,            nan,            nan],
        [1.71316910e+01, 1.45265424e+00, 1.99067421e+01, ...,
                    nan,            nan,            nan],
        [1.72732754e+01, 1.48946607e+00, 3.05821934e+01, ...,
                    nan,            nan,            nan],
        ...,
        [4.89881439e+01, 3.51128888e+00, 6.22970619e+01, ...,
         2.32764478e+03,            nan, 3.08653617e+00],
        [4.84218063e+01, 3.51128888e+00, 5.83327026e+01, ...,
         2.32764478e+03,            nan, 3.05821943e+00],
        [4.81386375e+01, 3.48297215e+00, 5.72000275e+01, ...,
         2.32198145e+03,            nan, 2.97326875e+00]],

       [[1.96000004e+01, 1.88000000e+00, 2.29799995e+01, ...,
                    nan,            nan,            nan],
        [1.96399994e+01, 1.81999993e+00, 2.30000000e+01, ...,
                    nan,            nan,            nan],
        [1.96800003e+01, 1.73000002e+00, 2.30399990e+01, ...,
                    nan,            nan,            nan],
        ...,
        [4.15699997e+01, 3.26999998e+00, 5.75699997e+01, ...,
         1.29408997e+03,            nan, 1.59999990e+00],
        [4.12199974e+01, 3.28999996e+00, 5.69899979e+01, ...,
         1.29563000e+03,            nan, 1.59999990e+00],
        [4.08899994e+01, 3.27999997e+00, 5.64199982e+01, ...,
         1.29753003e+03,            nan, 1.59999990e+00]]])
Coordinates:
  * location_index         (location_index) int64 59kB 0 1 2 ... 7405 7406 7407
  * value_time             (value_time) datetime64[ns] 210kB 2016-01-01 ... 2...
    reference_time         (value_time) datetime64[ns] 210kB 2016-01-01 ... 2...
    primary_location_id    (location_index) <U20 593kB 'usgs-01122500' ... 'u...
    secondary_location_id  (location_index) <U16 474kB 'nwm21-6163185' ... 'n...
  * timeseries_name        (timeseries_name) object 16B 'primary_value' 'seco...

In [60]:
# da_both.assign_attrs(measurement_unit="cms", configuration="nwm20_retrospective")
ds = da_both.to_dataset().chunk({"value_time": 8764, "location_index": 100}) 
ds = ds.assign_attrs(measurement_unit="cms", configuration="nwm21_retrospective")
ds = ds.set_xindex("primary_location_id")
ds = ds.set_xindex("secondary_location_id")
ds

<xarray.Dataset> Size: 3GB
Dimensions:                (location_index: 7408, value_time: 26280,
                            timeseries_name: 2)
Coordinates:
  * location_index         (location_index) int64 59kB 0 1 2 ... 7405 7406 7407
  * value_time             (value_time) datetime64[ns] 210kB 2016-01-01 ... 2...
    reference_time         (value_time) datetime64[ns] 210kB dask.array<chunksize=(8764,), meta=np.ndarray>
  * primary_location_id    (location_index) <U20 593kB 'usgs-01122500' ... 'u...
  * secondary_location_id  (location_index) <U16 474kB 'nwm21-6163185' ... 'n...
  * timeseries_name        (timeseries_name) object 16B 'primary_value' 'seco...
Data variables:
    streamflow             (timeseries_name, value_time, location_index) float64 3GB dask.array<chunksize=(2, 8764, 100), meta=np.ndarray>
Attributes:
    measurement_unit:  cms
    configuration:     nwm21_retrospective

In [61]:
%%time
# TODO: TEMP!
indx = 0

if indx == 0:
    ds.to_zarr(LOCAL_ZARR_JOINED_NWM21, mode="w", consolidated=True) 
else:
    ds.to_zarr(LOCAL_ZARR_JOINED_NWM21, mode="a", append_dim="timeseries_name", consolidated=True)

CPU times: user 5.97 s, sys: 1.15 s, total: 7.11 s
Wall time: 9.2 s


**Concat both configurations?**

In [104]:
ds_20 = xr.open_zarr(LOCAL_ZARR_JOINED_NWM20)
ds_21 = xr.open_zarr(LOCAL_ZARR_JOINED_NWM21)

In [105]:
ds_20 = ds_20.reset_coords("configuration", drop=True)
ds_20 = ds_20.reset_coords("primary_location_id", drop=True)
ds_20 = ds_20.reset_coords("secondary_location_id", drop=True)

ds_21 = ds_21.reset_coords("configuration", drop=True)
ds_21 = ds_21.reset_coords("primary_location_id", drop=True)
ds_21 = ds_21.reset_coords("secondary_location_id", drop=True)

In [81]:
# ds_20

In [82]:
# ds_21

In [90]:
ds_two = xr.open_mfdataset([LOCAL_ZARR_JOINED_NWM20, LOCAL_ZARR_JOINED_NWM21], engine="zarr", concat_dim="configuration_index", combine='nested')

In [91]:
ds_two

<xarray.Dataset> Size: 6GB
Dimensions:                (location_index: 7408, timeseries_name: 2,
                            value_time: 26280, configuration_index: 2)
Coordinates:
  * location_index         (location_index) int64 59kB 0 1 2 ... 7405 7406 7407
  * timeseries_name        (timeseries_name) object 16B 'primary_value' 'seco...
  * value_time             (value_time) datetime64[ns] 210kB 2016-01-01 ... 2...
    configuration          (configuration_index, timeseries_name) <U5 80B 'nw...
    primary_location_id    (configuration_index, location_index) object 119kB ...
    reference_time         (value_time) datetime64[ns] 210kB dask.array<chunksize=(8764,), meta=np.ndarray>
    secondary_location_id  (configuration_index, location_index) object 119kB ...
Dimensions without coordinates: configuration_index
Data variables:
    streamflow             (configuration_index, timeseries_name, value_time, location_index) float64 6GB dask.array<chunksize=(1, 2, 8764, 100), meta=np.ndarray>
Attributes:
    measurement_unit:  cms

In [106]:
# ds_two = ds_two.set_xindex("primary_location_id")
# ds_two = ds_two.set_xindex("secondary_location_id")
# ds_two = ds_two.set_xindex("configuration")

In [107]:
ds_two = xr.concat([ds_21, ds_20], pd.Index(["nwm21", "nwm20"], name="configuration"))  # , pd.Index(["nwm21", "nwm20"], name="configuration")

In [108]:
ds_two

<xarray.Dataset> Size: 6GB
Dimensions:          (location_index: 7408, timeseries_name: 2,
                      value_time: 26280, configuration: 2)
Coordinates:
  * location_index   (location_index) int64 59kB 0 1 2 3 ... 7404 7405 7406 7407
  * timeseries_name  (timeseries_name) object 16B 'primary_value' 'secondary_...
  * value_time       (value_time) datetime64[ns] 210kB 2016-01-01 ... 2018-12...
    reference_time   (value_time) datetime64[ns] 210kB 2016-01-01 ... 2018-12...
  * configuration    (configuration) object 16B 'nwm21' 'nwm20'
Data variables:
    streamflow       (configuration, timeseries_name, value_time, location_index) float64 6GB dask.array<chunksize=(1, 2, 8764, 100), meta=np.ndarray>
Attributes:
    measurement_unit:  cms

In [65]:
# ds_two = ds_two.reset_coords(names=["primary_location_id", "secondary_location_id"], drop=True)

In [114]:
pri_location_ids.shape

(7408,)

In [116]:
ds_two = ds_two.assign_coords(primary_location_id=("location_index", pd.Index(pri_location_ids)))
ds_two = ds_two.assign_coords(secondary_location_id=("location_index", pd.Index(sec_location_ids)))
ds_two = ds_two.set_xindex("primary_location_id")
ds_two = ds_two.set_xindex("secondary_location_id")
ds_two

<xarray.Dataset> Size: 6GB
Dimensions:                (location_index: 7408, timeseries_name: 2,
                            value_time: 26280, configuration: 2)
Coordinates:
  * location_index         (location_index) int64 59kB 0 1 2 ... 7405 7406 7407
  * timeseries_name        (timeseries_name) object 16B 'primary_value' 'seco...
  * value_time             (value_time) datetime64[ns] 210kB 2016-01-01 ... 2...
    reference_time         (value_time) datetime64[ns] 210kB 2016-01-01 ... 2...
  * configuration          (configuration) object 16B 'nwm21' 'nwm20'
  * primary_location_id    (location_index) object 59kB 'usgs-01115170' ... '...
  * secondary_location_id  (location_index) object 59kB 'nwm21-5862629' ... '...
Data variables:
    streamflow             (configuration, timeseries_name, value_time, location_index) float64 6GB dask.array<chunksize=(1, 2, 8764, 100), meta=np.ndarray>
Attributes:
    measurement_unit:  cms

In [71]:
ds_two = ds_two.chunk({"value_time": 8764, "location_index": 100}) 
ds_two = ds_two.assign_attrs(measurement_unit="cms")

In [80]:
ds_two.primary_location_id.values

array(['usgs-01010000', 'usgs-01010070', 'usgs-01010500', ...,
       'usgs-272524080221800', 'usgs-330304090210100',
       'usgs-385202111121601'], dtype=object)

In [121]:
ds_two.streamflow = ds_two.streamflow.to_numpy()

AttributeError: cannot set attribute 'streamflow' on a 'Dataset' object. Use __setitem__ styleassignment (e.g., `ds['name'] = ...`) instead of assigning variables.

In [119]:
ds_two.to_zarr(LOCAL_ZARR_JOINED_ALL, mode="w", consolidated=True) 

ValueError: buffer source array is read-only

# Rechunking

**If necessary**

In [1]:
import zarr
from rechunker import rechunk

In [7]:
source = zarr.open(LOCAL_ZARR_JOINED_ALL)
print(source.tree())

/
 ├── configuration (2,) object
 ├── primary_location_id (7386,) object
 ├── reference_time (26280,) int64
 ├── secondary_location_id (7386,) object
 ├── streamflow (2, 2, 26280, 7386) float64
 ├── timeseries_name (2,) object
 └── value_time (26280,) int64


In [9]:
source_arr = source["streamflow"]
source_arr.info

Name,/streamflow
Type,zarr.core.Array
Data type,float64
Shape,"(2, 2, 26280, 7386)"
Chunk shape,"(1, 2, 8764, 100)"
Order,C
Read-only,False
Compressor,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)"
Store type,zarr.storage.DirectoryStore
No. bytes,6211330560 (5.8G)
No. bytes stored,2655355394 (2.5G)


In [ ]:
target_chunks = {"RAINRATE": {"value_time": 1000, "location_id": 1000}, "location_id": None, "reference_time": None, "value_time": None}
max_mem = 134217216

# NOTE: Need to make sure these don't already exist
temp_store = "/data/merit_basins/tmp_rechunked.zarr"
target_store = "/data/merit_basins/test_merit_rechunked.zarr"

In [ ]:
array_plan = rechunk(
    source, target_chunks, max_mem, target_store, temp_store=temp_store
)
array_plan

In [ ]:
%%time
array_plan.execute()

In [ ]:
zarr.consolidate_metadata(target_store)

# Verify/Selections

In [28]:
ds_nwm20 = xr.open_zarr(LOCAL_ZARR_JOINED_NWM20)   # chunks={} to read variable as a dask array. This is default in .open_zarr()
ds_nwm20

<xarray.Dataset> Size: 3GB
Dimensions:                (location_index: 7386, value_time: 26280,
                            timeseries_name: 2)
Coordinates:
  * location_index         (location_index) int64 59kB 0 1 2 ... 7383 7384 7385
    primary_location_id    (location_index) <U20 591kB dask.array<chunksize=(3693,), meta=np.ndarray>
    reference_time         (value_time) datetime64[ns] 210kB dask.array<chunksize=(8764,), meta=np.ndarray>
    secondary_location_id  (location_index) <U16 473kB dask.array<chunksize=(3693,), meta=np.ndarray>
  * timeseries_name        (timeseries_name) object 16B 'primary_value' 'seco...
  * value_time             (value_time) datetime64[ns] 210kB 2016-01-01 ... 2...
Data variables:
    streamflow             (timeseries_name, value_time, location_index) float64 3GB dask.array<chunksize=(2, 8764, 100), meta=np.ndarray>
Attributes:
    configuration:     nwm20
    measurement_unit:  cms

In [30]:
def query_data_duckdb(filepath):
    """Load a parquet file(s) as a dataframe using duckdb"""
    query = f"""
        SELECT
            primary_location_id,
            value_time,
            primary_value,
        FROM
            read_parquet('{str(filepath)}')
    ;"""
    df = duckdb.sql(query).to_df()

    return df

df = query_data_duckdb(LOCAL_JOINED_NWM20_ALL)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [30]:
ds_zarr = ds_zarr.set_xindex(["primary_location_id", "secondary_location_id"])

In [93]:
ds_zarr.streamflow.sel(configuration="nwm21", primary_location_id="usgs-01010000").values.shape

(2, 26280, 1)

# Metrics

In [36]:
# import flox.xarray
from flox.xarray import xarray_reduce

In [82]:
ds_nwm20 = xr.open_zarr(LOCAL_ZARR_NWM20)
ds_nwm20

<xarray.Dataset> Size: 2GB
Dimensions:         (location_id: 7541, value_time: 26304)
Coordinates:
    configuration   <U19 76B ...
  * location_id     (location_id) <U16 483kB 'nwm20-10010638' ... 'nwm20-9998...
    reference_time  (value_time) datetime64[ns] 210kB dask.array<chunksize=(8764,), meta=np.ndarray>
  * value_time      (value_time) datetime64[ns] 210kB 2016-12-07T18:00:00 ......
Data variables:
    streamflow      (value_time, location_id) float64 2GB dask.array<chunksize=(8764, 100), meta=np.ndarray>
Attributes:
    measurement_unit:  cms

In [85]:
# xarray_reduce(ds_nwm20.streamflow, "location_id", func="mean", expected_groups=ds_nwm20.location_id.values)   # ("configuration", "primary_location_id")

<xarray.DataArray 'streamflow' (value_time: 26304, location_id: 7541)> Size: 2GB
dask.array<groupby_nanmean, shape=(26304, 7541), dtype=float64, chunksize=(8764, 100), chunktype=numpy.ndarray>
Coordinates:
    configuration   <U19 76B ...
    reference_time  (value_time) datetime64[ns] 210kB dask.array<chunksize=(8764,), meta=np.ndarray>
  * value_time      (value_time) datetime64[ns] 210kB 2016-12-07T18:00:00 ......
  * location_id     (location_id) object 60kB 'nwm20-10010638' ... 'nwm20-999...

# Random

In [55]:
USGS_TIMESERIES = Path(DATA_DIR, "usgs_2018.parquet")

In [53]:
query = f"SELECT DISTINCT location_id FROM read_parquet('{str(USGS_TIMESERIES)}');"

ids = duckdb.sql(query).fetchnumpy()

In [54]:
ids["location_id"].shape

(7375,)

In [28]:
def load_dataframe_duckdb(filepath):
    """Load a parquet file(s) as a pandas dataframe using duckdb"""
    query = f"""
        SELECT
            location_id,
            reference_time,
            value_time,
            value,
            CAST(variable_name as VARIABLE_NAME_DTYPE) as variable_name,
            CAST(measurement_unit as MEASUREMENT_UNIT_DTYPE) as measurement_unit,
            CAST(configuration as CONFIGURATION_DTYPE) as configuration
        FROM
            read_parquet('{str(filepath)}')
        LIMIT 100
    ;"""
    return duckdb.sql(query).to_df()  # WHERE value_time < '2016-06-01 00:00'

In [29]:
%%time
df2 = load_dataframe_duckdb(USGS_TIMESERIES)
df2

CPU times: user 85.1 ms, sys: 16.5 ms, total: 102 ms
Wall time: 46.1 ms


,location_id,reference_time,value_time,value,variable_name,measurement_unit,configuration
0,usgs-01010000,2016-01-01 05:00:00,2016-01-01 05:00:00,64.845581,streamflow,cms,usgs_gage_data
1,usgs-01010000,2016-01-01 17:00:00,2016-01-01 17:00:00,61.164387,streamflow,cms,usgs_gage_data
2,usgs-01010000,2016-01-02 05:00:00,2016-01-02 05:00:00,58.049534,streamflow,cms,usgs_gage_data
3,usgs-01010000,2016-01-02 17:00:00,2016-01-02 17:00:00,55.217850,streamflow,cms,usgs_gage_data
4,usgs-01010000,2016-01-03 05:00:00,2016-01-03 05:00:00,52.669334,streamflow,cms,usgs_gage_data
...,...,...,...,...,...,...,...
95,usgs-01010000,2016-02-17 17:00:00,2016-02-17 17:00:00,22.964962,streamflow,cms,usgs_gage_data
96,usgs-01010000,2016-02-18 05:00:00,2016-02-18 05:00:00,25.088726,streamflow,cms,usgs_gage_data
97,usgs-01010000,2016-02-18 17:00:00,2016-02-18 17:00:00,28.600014,streamflow,cms,usgs_gage_data
98,usgs-01010000,2016-02-19 05:00:00,2016-02-19 05:00:00,32.281204,streamflow,cms,usgs_gage_data


In [63]:
import numpy as np
import xarray as xr

from flox.xarray import xarray_reduce

arr = np.ones((4, 12))
labels1 = np.array(["a", "a", "c", "c", "c", "b", "b", "c", "c", "b", "b", "f"])
labels2 = np.array([1, 2, 2, 1])

da = xr.DataArray(
    arr, dims=("x", "y"), coords={"labels2": ("x", labels2), "labels1": ("y", labels1)}
)
da

<xarray.DataArray (x: 4, y: 12)> Size: 384B
array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])
Coordinates:
    labels2  (x) int64 32B 1 2 2 1
    labels1  (y) <U1 48B 'a' 'a' 'c' 'c' 'c' 'b' 'b' 'c' 'c' 'b' 'b' 'f'
Dimensions without coordinates: x, y

In [61]:
xarray_reduce(da, "labels1", "labels2", func="sum")

<xarray.DataArray (labels1: 4, labels2: 2)> Size: 64B
array([[ 4.,  4.],
       [ 8.,  8.],
       [10., 10.],
       [ 2.,  2.]])
Coordinates:
  * labels1  (labels1) object 32B 'a' 'b' 'c' 'f'
  * labels2  (labels2) int64 16B 1 2